In [1]:
import itertools as it
import pandas as pd
import numpy as np
import pdb
import time

In [2]:
def pretty_print(matrix, grid=False):
    print("----------------------------------------------------")
    if not len(matrix.shape) == 3:
        matrix = to_oh(matrix)
    if not grid:
        for pos in it.product(range(9), range(9)):
            i, j = pos
            print("{}, {}: {}".format(i+1, j+1, np.nonzero(matrix[i, j, :])[0]+1))
    else:
        for i in range(9):
            string = ""
            for j in range(9):
                possibilities = np.nonzero(matrix[i, j, :])[0]+1
                string += "{}".format(possibilities)
                if j <8: string += " , "
            print(string)

def to_oh(matrix):
    oh_matrix = np.zeros((9, 9, 9)).astype(np.int32)
    for i, j in it.product(range(9), range(9)):
        val = matrix[i][j]-1
        if val >= 0:
            oh_matrix[i][j][val] = 1
        else:
            oh_matrix[i][j] = np.ones((9,)).astype(np.int32)
    return oh_matrix

def from_oh(oh_matrix):
    matrix = np.zeros((9, 9), dtype=np.int32)
    for i, j in it.product(range(9), range(9)):
        possibilites = np.nonzero(oh_matrix[i, j, :])[0]
        if len(possibilites) == 1:
            matrix[i, j] = possibilites[0] + 1
        else:
            matrix[i, j] = 0
    return matrix

def grid_idxs():
    
    for grid_row in range(3):
        for grid_col in range(3):
            yield grid_row, grid_col
            
def check_solution(puzzle, solution):
        
    assert np.all(np.equal(solution[puzzle != 0], puzzle[puzzle != 0]))
    
    for row in range(9):
        assert len(np.setdiff1d(solution[row, :], np.arange(1, 10))) == 0
        
    for col in range(9):
        assert len(np.setdiff1d(solution[:, col], np.arange(1, 10))) == 0
        
    for grid_row, grid_col in grid_idxs():
        assert len(np.setdiff1d(solution[3*grid_row:3*grid_row+3, 3*grid_col:3*grid_col+3], np.arange(1, 10))) == 0

In [3]:
def resolve_row_constraints(solution):
            
    for row in range(9):
        
        grouped = np.zeros((9,))
        for group_size in range(1,5):
            ungrouped = np.where(grouped==0)[0]
            for group in it.combinations(ungrouped, group_size):
                nongroup = np.setdiff1d(np.arange(0,9), group)
                possibilities = set([])
                for col in group: 
                    possibilities.update(tuple(np.nonzero(solution[row, col, :])[0].tolist()))
                if len(possibilities) == group_size:
                    for col in group: 
                        grouped[col] = 1
                    for possibility in possibilities:
                        for col in nongroup:
                            solution[row, col, possibility] = 0
                            
        grouped = np.zeros((9,))
        for group_size in range(1,5):
            ungrouped = np.where(grouped==0)[0]
            for group in it.combinations(ungrouped, group_size):  
                nongroup = np.setdiff1d(np.arange(0,9), group)
                possibilities = set([])
                for val in group: 
                    possibilities.update(tuple(np.nonzero(solution[row, :, val])[0].tolist()))
                if len(possibilities) == group_size:
                    for val in group: 
                        grouped[val] = 1
                    for possibility in possibilities:
                        for val in nongroup:
                            solution[row, possibility, val] = 0
                            
    return solution

def resolve_col_constraints(solution):
            
    for col in range(9):
        
        grouped = np.zeros((9,))
        for group_size in range(1,5):
            ungrouped = np.where(grouped==0)[0]
            for group in it.combinations(ungrouped, group_size):
                nongroup = np.setdiff1d(np.arange(0,9), group)
                possibilities = set([])
                for row in group: 
                    possibilities.update(tuple(np.nonzero(solution[row, col, :])[0].tolist()))
                if len(possibilities) == group_size:
                    for row in group: 
                        grouped[row] = 1
                    for possibility in possibilities:
                        for row in nongroup:
                            solution[row, col, possibility] = 0
                            
        grouped = np.zeros((9,))
        for group_size in range(1,5):
            ungrouped = np.where(grouped==0)[0]
            for group in it.combinations(ungrouped, group_size):    
                nongroup = np.setdiff1d(np.arange(0,9), group)
                possibilities = set([])
                for val in group: 
                    possibilities.update(tuple(np.nonzero(solution[:, col, val])[0].tolist()))
                if len(possibilities) == group_size:
                    for val in group: 
                        grouped[val] = 1
                    for possibility in possibilities:
                        for val in nongroup:
                            solution[possibility, col, val] = 0
                            
    return solution

def resolve_grid_constraints(solution):        
        
    for grid_row, grid_col in grid_idxs():
        
        locs = list(it.product(np.arange(3*grid_row, 3*grid_row+3), np.arange(3*grid_col, 3*grid_col+3)))        
        grouped = np.zeros((9,))
        for group_size in range(1,5):
            ungrouped = np.where(grouped==0)[0]
            for group in it.combinations(ungrouped, group_size):
                nongroup = np.setdiff1d(np.arange(0,9), group)
                possibilities = set([])
                for loc_idx in group:
                    row, col = locs[loc_idx]
                    possibilities.update(tuple(np.nonzero(solution[row, col, :])[0].tolist()))
                if len(possibilities) == group_size:
                    for loc_idx in group: 
                        grouped[loc_idx] = 1
                    for possibility in possibilities:
                        for loc_idx in nongroup:
                            row, col = locs[loc_idx]
                            solution[row, col, possibility] = 0
        
        grouped = np.zeros((9,))
        for group_size in range(1,5):
            ungrouped = np.where(grouped==0)[0]
            for group in it.combinations(ungrouped, group_size):                            
                nongroup = np.setdiff1d(np.arange(0,9), group)
                possibilities = set([])
                for val in group:
                    locs = np.nonzero(solution[3*grid_row:3*grid_row+3, 3*grid_col:3*grid_col+3, val])
                    locs = list(zip(locs[0], locs[1]))
                    for possibility in locs:
                        possibilities.add(possibility)
                if len(possibilities) == group_size:
                    for val in group: 
                        grouped[val] = 1
                    for possibility in possibilities:
                        row, col = possibility
                        for val in nongroup:
                            solution[3*grid_row+row, 3*grid_col+col, val] = 0
                
    return solution

def solve(puzzle, debug=False):
 
    if debug: import pdb; pdb.set_trace()
    
    solution = to_oh(puzzle)
    
    while True:
        
        solution_cache = np.copy(solution)
                        
        solution = resolve_row_constraints(solution)
        
        solution = resolve_col_constraints(solution)

        solution = resolve_grid_constraints(solution)
                        
        if np.all(np.equal(solution, solution_cache)): break
    
    solution = from_oh(solution)
                
    return solution

In [4]:
puzzle = pd.read_csv('puzzle.txt', header=None).values
puzzle

array([[0, 0, 6, 0, 5, 0, 0, 3, 0],
       [3, 0, 0, 1, 0, 0, 0, 0, 7],
       [0, 8, 5, 0, 0, 2, 0, 9, 0],
       [9, 0, 0, 4, 0, 0, 0, 2, 0],
       [2, 0, 0, 0, 9, 0, 0, 0, 6],
       [0, 5, 0, 0, 0, 3, 0, 0, 9],
       [0, 7, 0, 8, 0, 0, 2, 6, 0],
       [4, 0, 0, 0, 0, 6, 0, 0, 3],
       [0, 3, 0, 0, 1, 0, 7, 0, 0]])

In [5]:
a = time.time()
solution = solve(puzzle)
b = time.time()
print("Time: %.2f" % (b-a))
check_solution(puzzle, solution)
solution

Time: 0.51


array([[1, 4, 6, 9, 5, 7, 8, 3, 2],
       [3, 9, 2, 1, 6, 8, 5, 4, 7],
       [7, 8, 5, 3, 4, 2, 6, 9, 1],
       [9, 6, 7, 4, 8, 1, 3, 2, 5],
       [2, 1, 3, 7, 9, 5, 4, 8, 6],
       [8, 5, 4, 6, 2, 3, 1, 7, 9],
       [5, 7, 1, 8, 3, 9, 2, 6, 4],
       [4, 2, 8, 5, 7, 6, 9, 1, 3],
       [6, 3, 9, 2, 1, 4, 7, 5, 8]], dtype=int32)